In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder


In [5]:
df = pd.read_csv("drug_200.csv")

print("Dataset shape:", df.shape)
print("First few rows:")
display(df.head())

Dataset shape: (200, 6)
First few rows:


,Age,Sex,BP,Cholesterol,Na_to_K,Drug
0,23,F,HIGH,HIGH,25.355,drugY
1,47,M,LOW,HIGH,13.093,drugC
2,47,M,LOW,HIGH,10.114,drugC
3,28,F,NORMAL,HIGH,7.798,drugX
4,61,F,LOW,HIGH,18.043,drugY


In [6]:
# Encode categorical features
le = LabelEncoder()
for col in df.columns:
    if df[col].dtype == 'object':
        df[col] = le.fit_transform(df[col])

X = df.drop("Drug", axis=1).values
y = df["Drug"].values



In [36]:
def std_scaling(df):
    return (df-df.mean())/(df.std() + 1e-8)


In [37]:
X=std_scaling(X)

# healper function

In [ ]:
def sigmoid(z):
    # Clip to avoid overflow in exp()
    z = np.clip(z, -500, 500)
    return 1 / (1 + np.exp(-z))

def compute_cost(X, y, weights, bias, reg_type=None, reg_lambda=0.01, l1_ratio=0.5):
    m = len(y)
    z = np.dot(X, weights) + bias
    y_pred = sigmoid(z)
    epsilon = 1e-9  # to prevent log(0)

    # Binary cross-entropy loss
    cost = -(1/m) * np.sum(y*np.log(y_pred + epsilon) + (1 - y)*np.log(1 - y_pred + epsilon))
    
    # Regularization terms
    if reg_type == 'l2':  # Ridge
        cost += (reg_lambda / (2*m)) * np.sum(np.square(weights))
    elif reg_type == 'l1':  # Lasso
        cost += (reg_lambda / (2*m)) * np.sum(np.abs(weights))
    elif reg_type == 'elasticnet':  # Elastic Net
        l1_term = l1_ratio * np.sum(np.abs(weights))
        l2_term = (1 - l1_ratio) * np.sum(np.square(weights))
        cost += (reg_lambda / (2*m)) * (l1_term + l2_term)
    return cost

def update_weights(X, y, weights, bias, lr=0.01, reg_type=None, reg_lambda=0.01, l1_ratio=0.5):
    m = len(y)
    z = np.dot(X, weights) + bias
    y_pred = sigmoid(z)

    # Gradient computation
    error = (y_pred - y)
    dw = (1/m) * np.dot(X.T, error)
    db = (1/m) * np.sum(error)

    # Regularization (gradient)
    if reg_type == 'l2':  # Ridge
        dw += (reg_lambda / m) * weights
    elif reg_type == 'l1':  # Lasso
        dw += (reg_lambda / m) * np.sign(weights)
    elif reg_type == 'elasticnet':  # Elastic Net
        dw += (reg_lambda / m) * (l1_ratio * np.sign(weights) + (1 - l1_ratio) * weights)

    # Parameter update (learning rate tuned)
    weights -= lr * dw
    bias -= lr * db
    return weights, bias

def fit_logistic_regression(X, y, lr=0.05, epochs=3000, reg_type=None, reg_lambda=0.01, l1_ratio=0.5):
    m, n = X.shape
    weights = np.zeros(n)
    bias = 0

    # Normalize features (important for stable convergence)
    X = (X - np.mean(X, axis=0)) / (np.std(X, axis=0) + 1e-8)

    for i in range(epochs):
        weights, bias = update_weights(X, y, weights, bias, lr, reg_type, reg_lambda, l1_ratio)

        # Optional: early stopping every 500 iterations (for faster convergence)
        if i % 500 == 0:
            cost = compute_cost(X, y, weights, bias, reg_type, reg_lambda, l1_ratio)
            # print(f"Epoch {i}, Cost: {cost:.4f}")
    return weights, bias

def predict(X, weights, bias):
    X = (X - np.mean(X, axis=0)) / (np.std(X, axis=0) + 1e-8)
    z = np.dot(X, weights) + bias
    y_pred = sigmoid(z)
    return (y_pred >= 0.5).astype(int)



## Evaluation metrics

In [ ]:

def evaluate(y_true, y_pred, name="Model"):
    accuracy = np.mean(y_true == y_pred)
    precision_list, recall_list, f1_list = [], [], []

    for c in np.unique(y_true):
        tp = np.sum((y_pred == c) & (y_true == c))
        fp = np.sum((y_pred == c) & (y_true != c))
        fn = np.sum((y_pred != c) & (y_true == c))

        precision = tp / (tp + fp + 1e-9)
        recall = tp / (tp + fn + 1e-9)
        f1 = 2 * precision * recall / (precision + recall + 1e-9)

        precision_list.append(precision)
        recall_list.append(recall)
        f1_list.append(f1)

    accuracy_score,precision_score,recall_score,f1_score=accuracy,np.mean(precision_list),np.mean(recall_list),np.mean(f1_list)

    return accuracy_score,precision_score,recall_score,f1_score


## Cross-validation

In [44]:
def cross_validate(X, y, reg_type=None, reg_lambda=0.01, l1_ratio=0.5, lr=0.1, epochs=2000):
    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    metrics = {'accuracy': [], 'precision': [], 'recall': [], 'f1': []}
    
    for train_idx, test_idx in kf.split(X):
        X_train, X_test = X[train_idx], X[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]
        
        weights, bias = fit_logistic_regression(
            X_train, y_train, lr=lr, epochs=epochs,
            reg_type=reg_type, reg_lambda=reg_lambda, l1_ratio=l1_ratio
        )
        y_pred = predict(X_test, weights, bias)

        accuracy_score,precision_score,recall_score,f1_score=evaluate(y_test,y_pred)
        
        metrics['accuracy'].append(accuracy_score)
        metrics['precision'].append(precision_score)
        metrics['recall'].append(recall_score)
        metrics['f1'].append(f1_score)
    
    return {k: np.mean(v) for k, v in metrics.items()}

# Evaluate Regularization

In [45]:
results = {}

results["No Regularization"] = cross_validate(X, y, reg_type=None, lr=0.5, epochs=5000)
results["Lasso (L1)"] = cross_validate(X, y, reg_type='l1', reg_lambda=0.3, lr=0.5, epochs=5000)
results["Ridge (L2)"] = cross_validate(X, y, reg_type='l2', reg_lambda=0.3, lr=0.5, epochs=5000)
results["Elastic Net"] = cross_validate(X, y, reg_type='elasticnet', reg_lambda=0.4, l1_ratio=0.5, lr=0.5, epochs=5000)

In [46]:
result_df = pd.DataFrame(results).T
display(result_df)

print("\nAverage metrics across 5 folds:")
print(result_df)

,accuracy,precision,recall,f1
No Regularization,0.08,0.0165,0.21,0.030395
Lasso (L1),0.08,0.0165,0.21,0.030395
Ridge (L2),0.08,0.0165,0.21,0.030395
Elastic Net,0.08,0.0165,0.21,0.030395



Average metrics across 5 folds:
                   accuracy  precision  recall        f1
No Regularization      0.08     0.0165    0.21  0.030395
Lasso (L1)             0.08     0.0165    0.21  0.030395
Ridge (L2)             0.08     0.0165    0.21  0.030395
Elastic Net            0.08     0.0165    0.21  0.030395
